In [3]:
import pandas as pd
import tweepy as tw
import re
import emoji

In [4]:
from textblob import TextBlob

In [5]:
import nltk
#nltk.download('words')
words = set(nltk.corpus.words.words())

In [6]:
columns1 = ['User ID', 'Item ID','Rating', 'Timestamp']
df_ratings = pd.read_table('u.data', names=columns1)


In [7]:
columns2 = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime' ,'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
df_movie = pd.read_csv('u.item', sep='|', names=columns2, encoding='latin-1')

In [8]:
df_ratings.head(3)

,User ID,Item ID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


In [9]:
df_movie.head(3)

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
access_key = '1076801107-YnSnzPG8QVLCHTpAIzH0B0iN5y4Ktg3ooWBtC32'
access_secret = 'tu6pZ59yae4PbBWl7VamRjUgLzaV3Dl1hWpjS96tvCCjk'
consumer_key = 'yUv9lg35fpu56VJPPgxx54KQI'
consumer_secret ='hGgAALXQrIiB86gSmFxHqj3Qm3RRHEmPHh1sq5UW702tR32k1D'

In [11]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [12]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI['en']) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.word_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet

In [13]:
def collect_tweets_polarity(search_text):
    tweets = tw.Cursor(api.search_tweets, 
                           q=search_text,
                           lang="en").items(10)

    users_locs = [[tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]
    tweet_text = pd.DataFrame(data=users_locs, columns=['user','location','tweet'])

    tweet_list = list(tweet_text.tweet)
    
    clean_tweet_list = []

    for i in tweet_list:
        clean_tweet_list.append(cleaner(i))

    polar_score = []

    for i in clean_tweet_list:
        res = TextBlob(i)
        polar_score.append(res.sentiment.polarity)

    if(len(polar_score)>0):
        avg = round(sum(polar_score)/len(polar_score),3)
    else:
        avg=0.000
    return avg

In [14]:
def score_generator(mov_list):
    scores=[]

    for i in range(len(mov_list)):

        sample = mov_list[i][:-7]
        sample=sample.lower()
        sample=sample.split(" ")
        sample = "#" + "".join(sample)
        scores.append(collect_tweets_polarity(sample))

    return scores


In [15]:
movie_list = list(df_movie.iloc[:,1].values)

In [16]:
movie_scores = score_generator(movie_list)

Rate limit reached. Sleeping for: 625
Rate limit reached. Sleeping for: 839


In [ ]:
save('movie scores.npy')